# Preparación

In [ ]:
pip install --upgrade scikit-learn

In [ ]:
pip install scikit-tda

In [ ]:
pip install gudhi

In [ ]:
import time
import warnings
from ripser import ripser
import numpy as np
import matplotlib.pyplot as plt
from persim import plot_diagrams
import persim
import glob
from numpy import asarray
from PIL import Image
import os
import csv

from sklearn.utils import shuffle
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
import random
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
import warnings

import pandas as pd

warnings.filterwarnings('ignore')

In [ ]:
!wget https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/EVb0cuOzAMNAmzMx9_U9EfEBj9ZSpd9vdPsA_hepzC5NqQ?download=1 -O bci.csv

In [ ]:
def load_prima():
  df=pd.read_csv("bci.csv", sep=',',header=None)
  df1 = df.iloc[:,:-1]
  df2=df.iloc[:,-1]
  return df1.values,df2.values

In [ ]:
def preparar_dataset(data,target,n_labeled=15, fold=0):
  numFolds=5
  scaler = StandardScaler()
  data=scaler.fit_transform(data)
  unos=list(np.where(target==1)[0])
  unos=shuffle(unos,random_state=10)

  ceros=list(np.where(target==0)[0])
  ceros=shuffle(ceros,random_state=10)

  k_ceros=len(ceros)//numFolds
  k_unos=len(unos)//numFolds

  fold_ceros=[ceros[k_ceros*i:k_ceros*(i+1)] for i in range(numFolds-1)]
  fold_unos=[unos[k_unos*i:k_unos*(i+1)] for i in range(numFolds-1)]

  fold_ceros.append(ceros[k_ceros*(numFolds-1):])
  fold_unos.append(unos[k_unos*(numFolds-1):])

  valid_ceros=fold_ceros[fold]
  valid_unos=fold_unos[fold]

  label_fold_ceros=fold_ceros[(fold+1)%numFolds][:n_labeled]
  label_fold_unos=fold_unos[(fold+1)%numFolds][:n_labeled]

  unlabeled_ceros=set()
  unlabeled_unos=set()

  for i in range(numFolds):
    if i!=fold and i!=(fold+1)%numFolds:
      unlabeled_ceros=unlabeled_ceros.union(set(fold_ceros[i]))
      unlabeled_unos=unlabeled_unos.union(set(fold_unos[i]))
    elif i==(fold+1)%numFolds:
      unlabeled_ceros=unlabeled_ceros.union(set(fold_ceros[i][n_labeled:]))
      unlabeled_unos=unlabeled_unos.union(set(fold_unos[i][n_labeled:]))


  puntos_ceros_valid=np.array(data)[valid_ceros]
  puntos_unos_valid=np.array(data)[valid_unos]

  puntos_ceros=np.array(data)[label_fold_ceros]
  puntos_unos=np.array(data)[label_fold_unos]

  X_unlabeled_ceros=np.array(data)[list(unlabeled_ceros)]
  X_unlabeled_unos=np.array(data)[list(unlabeled_unos)]


  print("SUMMARY")
  print("Train 0: "+str(len(puntos_ceros)))
  print("Train 1: "+str(len(puntos_unos)))
  print("Valid 0: "+str(len(puntos_ceros_valid)))
  print("Valid 1: "+str(len(puntos_unos_valid)))
  print("Unlabelled 0: "+str(len(X_unlabeled_ceros)))
  print("Unlabelled 1: "+str(len(X_unlabeled_unos)))

  return (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import scipy.spatial.distance as dist
from scipy.sparse.csgraph import minimum_spanning_tree

def compute_Mf_0(X, Z, is_dist=False):
    """Compute the matching induced by the inclusion X --> Z given by sending points from X, in order, to the first points from Z.
    We assume that coordinates of X and Z are given as numpy arrays, where the number of rows from Z is greater or equal to X.
    If is_dist is True, we assume that X and Z are given as distance matrices.

    This returns a pair of lists with endpoints of intervals, that are representations of both barcodes, together with a matching between such lists given by a list of indices.
"""
    filtration_list_X, pairs_arr_X = mst_edge_filtration(X, is_dist=is_dist) # MST(X)
    filtration_list_Z, pairs_arr_Z = mst_edge_filtration(Z, is_dist=is_dist) # MST(Z)
    TMT_X_pairs = compute_tmt_pairs(filtration_list_X, pairs_arr_X)
    TMT_Z_pairs = compute_tmt_pairs(filtration_list_Z, pairs_arr_Z)
    indices_X_Z = np.max(TMT_Z_pairs, axis=1)<X.shape[0]
    TMT_X_Z_pairs = TMT_Z_pairs[indices_X_Z]
    indices_X_Z = np.nonzero(indices_X_Z)[0]
    FX = get_inclusion_matrix(TMT_X_pairs, TMT_X_Z_pairs) # Associated matrix
    matchingX = get_inclusion_matrix_pivots(FX, Z.shape[0]) # Matching in TMT_X_Z
    matching =[indices_X_Z[i] for i in matchingX] # Matching in all TMT_Z
    return filtration_list_X, filtration_list_Z, matching

def read_csr_matrix(cs_matrix):
    """Function to read output from minimum_spanning_tree and prepare it as a list of
    filtration values (in order) together with an array with the corresponding pairs.
    """
    filtration_list = []
    edges = []
    entry_idx = 0
    for i, cummul_num_entries in enumerate(cs_matrix.indptr[1:]):
        while entry_idx < cummul_num_entries:
            edges.append((i, cs_matrix.indices[entry_idx]))
            filtration_list.append(cs_matrix.data[entry_idx])
            entry_idx+=1
    # Sort filtration values and pairs
    edges_arr = np.array(edges)
    np.argsort(filtration_list)
    sort_idx = np.argsort(filtration_list)
    filtration_list = np.array(filtration_list)[sort_idx].tolist()
    edges_arr = edges_arr[sort_idx]
    return filtration_list, edges_arr

def mst_edge_filtration(points, is_dist=False):
    """Returns the edges and filtration values for the Euclidean minimum spanning tree
    of a given point sample.
    This is a wrapper for the scipy minimum_spanning_tree function.
    """
    if is_dist:
        mst = minimum_spanning_tree(points)
    else:
        mst = minimum_spanning_tree(dist.squareform(dist.pdist(points)))
    # We now read the compressed sparse row matrix
    return read_csr_matrix(mst)

def compute_tmt_pairs(filtration_list, edges_arr, tolerance=10e-8):
    # Get proper merge tree pairs
    E_b = []
    C = np.array(list(range(edges_arr.shape[0]+1)))
    tmt_pairs_list = []
    for i, (b, edge) in enumerate(zip(filtration_list, edges_arr)):
        E_b.append(edge)
        # If the next filtration value is very close, we continue addin edges to E_b
        if i < len(filtration_list)-1:
            if np.abs(b - filtration_list[i+1]) < tolerance:
                continue

        # We iterate over E_b, adding triplets to tmt_pairs_list following steps (i)-(v) from the article
        while(len(E_b)>0):
            # (i) we take the edge [i,j] from E_b such that min{C[i],C[j]} is smallest
            E_b_C_min = [np.min(C[edge]) for edge in E_b]
            idx = np.argmin(E_b_C_min)
            edge = E_b[idx]
            # (ii)
            M, m = np.max(C[edge]), np.min(C[edge])
            assert m < M
            # (iii)
            tmt_pairs_list.append([M,m])
            # (iv)
            C[C==M]=m
            # (v)
            del(E_b[idx])
        # end when E_b is empty
    # end computing tmt
    tmt_pairs_arr = np.array(tmt_pairs_list)
    return tmt_pairs_arr


def add_columns_mod_2(col1, col2):
    """ Given two lists of integers, which are sparse representations of a pair of vectors in Z mod 2, this funciton adds them and
    returns the result in the same input format.
    """
    diff_1 = set(col1).difference(set(col2))
    diff_2 = set(col2).difference(set(col1))
    result = diff_1.union(diff_2)
    return list(result)


def get_inclusion_matrix(pairs_arr_X, pairs_arr_Z, subset_indices=[]):
    """ Given two pairs of arrays with the vertex merge pairs, this function returns the associated inclusion matrix.
    From the point of view of minimum spanning trees, the output matrix columns can be interpreted as the minimum paths that are needed to
    go through in MST(Z) in order to connect the endpoints from an edge in MST(X)
    """
    # If subset indices are not specified, we assume that the indices of vertices from S correspond to the first #X vertices from Z
    if (len(subset_indices)==0):
        subset_indices = list(range(pairs_arr_X.shape[0]+1))
    pivot2column = [-1] + np.argsort(np.max(pairs_arr_Z, axis=1)).tolist()
    inclusion_matrix = []
    for col_X in pairs_arr_X:
        col_X = [subset_indices[i] for i in col_X]
        col_M = []
        while(len(col_X)>0):
            piv = np.max(col_X)
            col_M.append(pivot2column[piv])
            col_X = add_columns_mod_2(col_X, pairs_arr_Z[pivot2column[piv]])
        # end reducing column X
        col_M.sort()
        inclusion_matrix.append(col_M)
    return inclusion_matrix

def get_inclusion_matrix_pivots(matrix_list, num_rows):
    """ Returns the pivots of a matrix given in list format"""
    pivots = []
    pivot2column = np.ones(num_rows, dtype="int")*-1
    for i, column in enumerate(matrix_list):
        reduce_column = list(column)
        piv = np.max(reduce_column)
        while(pivot2column[piv]>-1):
            reduce_column = add_columns_mod_2(reduce_column, matrix_list[pivot2column[piv]])
            piv = np.max(reduce_column)
            # we assume that columns are never reduced to the 0 column
        pivots.append(piv)
        pivot2column[piv] = i
    # end getting pivots
    return pivots


def plot_matching_0(filt_X, filt_Z, matching, ax):
    """ Given two zero dimensional barcodes as well as a block function between them, this function plots the associated diagram"""
    # Plot matching barcode
    for i, Z_end in enumerate(filt_Z):
        if i in matching:
            X_end = filt_X[matching.index(i)]
            ax.add_patch(mpl.patches.Rectangle([0, i-0.2], Z_end, 0.4, color="navy", zorder=2))
            ax.add_patch(mpl.patches.Rectangle([Z_end*0.9, i-0.2], X_end-Z_end, 0.4, color="orange", zorder=1.9))
        else:
            ax.add_patch(mpl.patches.Rectangle([0, i-0.2], Z_end, 0.4, color="aquamarine", zorder=2))

    MAX_PLOT_RAD = max(np.max(filt_X), np.max(filt_Z))*1.1
    ax.set_xlim([-0.1*MAX_PLOT_RAD, MAX_PLOT_RAD*1.1])
    ax.set_ylim([-0.1*len(filt_X), len(filt_Z)])
    ax.set_frame_on(False)
    ax.set_yticks([])
#end plot_matching_0

def plot_density_matrix(filt_X, filt_Z, matching, ax, nbins=5):
    endpoints_Z = np.array(filt_Z)[matching]
    differences = np.array([[a, b] for (a,b) in zip(filt_X, endpoints_Z)])
    ax.hist2d(differences[:,0], differences[:,1], bins=(nbins, nbins), cmap=plt.cm.jet)
    ax.set_xlabel('Ends in X')
    ax.set_ylabel('Ends in Z')

def plot_density_matrix_percentage(filt_X, filt_Z, matching, ax, nbins=5):
    ends_X = np.array(filt_X)
    max_end = np.max(ends_X)
    ends_diff = ends_X - np.array(filt_Z)[matching]
    Diag_diff = np.vstack((ends_X, ends_diff)).transpose()
    hist = np.histogram2d(Diag_diff[:,1], Diag_diff[:,0], bins=nbins, range=[[0,max_end], [0,max_end]])[0]
    sum_cols = np.sum(hist, axis=0)
    sum_cols = np.maximum(sum_cols, 1)
    hist = np.divide(hist, sum_cols)
    hist=hist[-1::-1]
    ax.imshow(hist, extent=(0, max_end, 0, max_end))
    ax.set_xlabel('Differences of the bars (percent)')
    ax.set_ylabel('Length of the bars Z')

def compute_matching_diagram(filt_X, filt_Z, matching, _tol=1e-5):
    pairs = []
    for i, a in enumerate(filt_X):
        b = filt_Z[matching[i]]
        pairs.append((a, b))
    # end for
    multiplicities = []
    pairs_copy = list(pairs)
    old_pair = pairs_copy.pop()
    pairs = [old_pair] # create new pairs list with non-repeating entries
    multiplicities.append(1)
    # First compute matched pairs and their multiplicities
    while len(pairs_copy)>0:
        pair = pairs_copy.pop()
        if np.all(np.abs(np.array(pair)-np.array(old_pair)) < _tol):
            multiplicities[-1]+=1
        else:
            multiplicities.append(1)
            pairs.append(pair)
            old_pair = pair
        # end checking if pair similar
    # end while
    # Next compute right infinity points and their multiplicities
    unmatched_idx = [j for j in range(len(filt_Z)) if j not in matching]
    if len(unmatched_idx)>0:
        b = filt_Z[unmatched_idx.pop()]
        pairs.append((np.inf, b))
        multiplicities.append(1)
        while len(unmatched_idx)>0:
            prev_b = b
            b = filt_Z[unmatched_idx.pop()]
            if (np.abs(b-prev_b)<_tol):
                multiplicities[-1]+=1
            else:
                pairs.append((np.inf, b))
                multiplicities.append(1)
            # end checking same endpoint
        # iterate over unmatched intervals
    # only if some intervals are unmatched
    return np.array(pairs), multiplicities


def plot_matching_diagram(pairs, ax, colorpt="black", marker="o", size=15, hmax=None):
    fin_pairs = pairs[pairs[:,0]<np.inf]
    if hmax is None:
        max_x = max(np.max(fin_pairs), np.max(pairs[:,1]))
        lim_x = max_x*1.3
        infty_x = max_x*1.1
    else:
        max_x, lim_x, infty_x = hmax, hmax*1.3, hmax*1.1
    # end if-else loop
    ax.plot([0, lim_x], [0, lim_x], c="gray", linewidth=1, zorder=1)
    ax.plot([infty_x, infty_x], [0, lim_x], c="blue", linewidth=1, zorder=1)
    ax.scatter(fin_pairs[:,0], fin_pairs[:,1], color=colorpt, s=size, marker=marker, zorder=2)
    inf_points = pairs[pairs[:,0]==np.inf]
    ax.scatter(np.ones(len(inf_points))*infty_x, inf_points[:,1], color=colorpt, s=size, marker=marker, zorder=2)
    ax.text(infty_x*1.02, infty_x*1.1, "∞", fontsize=15, color="blue")
    ax.scatter([infty_x], [infty_x], color=colorpt, s=size, marker=marker, zorder=2)
    # Adjust spines
    ax.spines[["top", "right"]].set_visible(False)
    ax.spines[["bottom","left"]].set_position("zero")
    # adjust margins
    ax.set_xlim([0,lim_x])
    ax.set_ylim([0,lim_x])

def plot_density_matching_diagram(D_f_rep, coker_f, savepath, nbins=5, cmap="jet", show_colorbar=False):
    ### First, create figure with two axes usin gridspec
    gs = mpl.gridspec.GridSpec(1,2, width_ratios=[nbins,1])
    fig = plt.figure(figsize=(4,4))
    ax = [fig.add_subplot(gs[0,i]) for i in range(2)]
    ### Next, let us read the information from the matching diagram and plot it
    fin_D_f = D_f_rep[D_f_rep[:,0]<np.inf]
    lim_x = 1.2*max(np.max(fin_D_f), np.max(D_f_rep[:,1]))
    hist_fin_D_f = np.histogram2d(fin_D_f[:,1], fin_D_f[:,0], bins=nbins, range=[[0,lim_x], [0,lim_x]])[0]
    hist_coker_f = np.histogram2d(np.zeros(len(coker_f)), coker_f, bins=[1,nbins], range=[[0,lim_x*0.1], [0,lim_x]])[0]
    vmax = max(np.max(hist_fin_D_f), np.max(hist_coker_f))
    ax[0].imshow(hist_fin_D_f, cmap=cmap, vmin=0, vmax=vmax, origin="lower", extent=(0,lim_x,0,lim_x))
    image = ax[1].imshow(hist_coker_f.transpose(), cmap=cmap, vmin=0, vmax=vmax, origin="lower", extent=(0,lim_x/nbins,0,lim_x))
    ax[1].set_yticks([])
    ax[1].set_xticks([])
    # Draw labels
    ax[0].set_xlabel('Ends in X')
    ax[0].set_ylabel('Ends in Z')
    ax[1].set_ylabel('Coker(f)')
    # Colorbar
    if show_colorbar:
        plt.colorbar(mappable=image, ax=ax, orientation="horizontal", location="top")
    ### Save figure
    plt.savefig(savepath)

def plot_matching_0(filt_X, filt_Z, matching, ax):
    """ Given two zero dimensional barcode endpoint lists as well as a block function between them, this function plots the associated diagram"""
    # Plot matching barcode
    for i, Z_end in enumerate(filt_Z):
        if i in matching:
            X_end = filt_X[matching.index(i)]
            ax.add_patch(mpl.patches.Rectangle([0, i-0.2], Z_end, 0.4, color="navy", zorder=2))
            ax.add_patch(mpl.patches.Rectangle([Z_end*0.9, i-0.2], X_end-Z_end, 0.4, color="orange", zorder=1.9))
        else:
            ax.add_patch(mpl.patches.Rectangle([0, i-0.2], Z_end, 0.4, color="aquamarine", zorder=2))

    MAX_PLOT_RAD = max(np.max(filt_X), np.max(filt_Z))*1.1
    ax.set_xlim([-0.1*MAX_PLOT_RAD, MAX_PLOT_RAD*1.1])
    ax.set_ylim([-0.1*len(filt_Z), len(filt_Z)])
    ax.set_frame_on(False)
    ax.set_yticks([])
#end plot_matching_0

### Random Circle Creation
def sampled_circle(r, R, n, RandGen):
    assert r<=R
    radii = RandGen.uniform(r,R,n)
    angles = RandGen.uniform(0,2*np.pi,n)
    return np.vstack((np.cos(angles)*radii, np.sin(angles)*radii)).transpose()

In [ ]:
def calc_dist(punto,conjunto):
  z=np.append(conjunto,np.expand_dims(punto,axis=0),axis=0)
  a,b,c=compute_Mf_0(conjunto,z)
  D_f, multiplicities = compute_matching_diagram(a, b, c, _tol=1e-5)
  D_f_rep = []
  for i, pair in enumerate(D_f):
      for j in range(multiplicities[i]):
          D_f_rep += list(pair)

  D_f_rep = np.array(D_f_rep).reshape(-1,2)

  fin_D_f = D_f_rep[D_f_rep[:,0]<np.inf]

  coker_f = D_f[D_f[:,0]==np.inf][:,1]

  distancia=np.sum(coker_f)
  return distancia

In [ ]:
def analizar_punto_rocio(punto,puntos_ceros,puntos_unos,th=0):

  d0=calc_dist(punto,puntos_ceros)

  d1=calc_dist(punto,puntos_unos)

  d=d0+d1


  if max(d0/d,d1/d)<th:
    clase=-1
  else:
    if d0>d1:
      clase=1
    else:
      clase=0
  return clase

In [ ]:
def analizar_puntos_rocio(data,target,func,th=0,reduccion=False,dim=0,fold=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target,fold=fold)
  embedding=data
  if reduccion==True:
    embedding=umap.UMAP(random_state=75).fit_transform(data)
  (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target,fold=fold)


  bien=0
  mal=0
  dudoso=0

  puntos_ceros_final=puntos_ceros.tolist()
  puntos_unos_final=puntos_unos.tolist()
  puntos_dudosos_final=[]

  for i,punto in enumerate(X_unlabeled_unos_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,th)
    if clase==1:
      bien=bien+1
      puntos_unos_final.append(X_unlabeled_unos[i])
    elif clase==0:
      mal=mal+1
      puntos_ceros_final.append(X_unlabeled_unos[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_unos[i])



  for i,punto in enumerate(X_unlabeled_ceros_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,th)
    if clase==1:
      mal=mal+1
      puntos_unos_final.append(X_unlabeled_ceros[i])
    elif clase==0:
      bien=bien+1
      puntos_ceros_final.append(X_unlabeled_ceros[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_ceros[i])

  metrics_svm,metrics_rf=comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

  return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final),metrics_svm,metrics_rf)

In [ ]:
def analizar_punto(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.bottleneck(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.bottleneck(dgms_unos_mod, dgms_unos, matching=False)
  clase=-1

  if distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_punto_th(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0.8,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.bottleneck(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.bottleneck(dgms_unos_mod, dgms_unos, matching=False)

  distancia=distance_bottleneck_cero+distance_bottleneck_uno

  if max(distance_bottleneck_cero/distancia,distance_bottleneck_uno/distancia)<th:
    clase=-1
  elif distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_punto_Wasserstein(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.wasserstein(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.wasserstein(dgms_unos_mod, dgms_unos, matching=False)
  clase=-1

  if distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_punto_Wasserstein_th(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0.8,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.wasserstein(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.wasserstein(dgms_unos_mod, dgms_unos, matching=False)

  distancia=distance_bottleneck_cero+distance_bottleneck_uno

  if max(distance_bottleneck_cero/distancia,distance_bottleneck_uno/distancia)<th:
    clase=-1
  elif distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_puntos(data,target,func,th=0,reduccion=False,dim=0,fold=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target,fold=fold)
  embedding=data
  if reduccion==True:
    embedding=umap.UMAP(random_state=75).fit_transform(data)
  (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target,fold=fold)


  bien=0
  mal=0
  dudoso=0

  puntos_ceros_final=puntos_ceros.tolist()
  puntos_unos_final=puntos_unos.tolist()
  puntos_dudosos_final=[]

  dgms_ceros = ripser(puntos_ceros_umap)['dgms'][dim]
  dgms_unos = ripser(puntos_unos_umap)['dgms'][dim]
  for i,punto in enumerate(X_unlabeled_unos_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,dgms_ceros,dgms_unos,th,dim)
    if clase==1:
      bien=bien+1
      puntos_unos_final.append(X_unlabeled_unos[i])
    elif clase==0:
      mal=mal+1
      puntos_ceros_final.append(X_unlabeled_unos[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_unos[i])



  for i,punto in enumerate(X_unlabeled_ceros_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,dgms_ceros,dgms_unos,th,dim)
    if clase==1:
      mal=mal+1
      puntos_unos_final.append(X_unlabeled_ceros[i])
    elif clase==0:
      bien=bien+1
      puntos_ceros_final.append(X_unlabeled_ceros[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_ceros[i])

  metrics_svm,metrics_rf=comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

  return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final),metrics_svm,metrics_rf)

In [ ]:
def resumen(bien,mal,dudosos):
  print('-----------------RESUMEN------------------')
  print('Correct '+str(bien))
  print('Wrong '+str(mal))
  print('Unlabelled '+str(dudosos))
  print('')
  acc=0
  if bien+mal!=0:
    print('Correct percentage '+str(bien/(bien+mal)))
    acc=bien/(bien+mal)
  else:
    print('Correct percentage 0')
  print('Labelled percentage '+str((bien+mal)/(bien+mal+dudosos)))
  anotados=(bien+mal)/(bien+mal+dudosos)
  return acc,anotados

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


def comprobar_accuracy(puntos_ceros_final,puntos_unos_final,X_unlabeled_ceros_valid,X_unlabeled_unos_valid):
    datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelSVMLineal = SVC(kernel="linear",probability=True,random_state=90,max_iter=10000)
    modelSVMLineal.fit(datos, labels)

    predicted_ceros=np.array(modelSVMLineal.predict(X_unlabeled_ceros_valid))
    target_ceros=np.zeros_like(predicted_ceros)



    predicted_unos=np.array(modelSVMLineal.predict(X_unlabeled_unos_valid))
    target_unos=np.ones_like(predicted_unos)


    predicted=np.concatenate((predicted_ceros,predicted_unos))
    target=np.concatenate((target_ceros,target_unos))

    acc_svm=accuracy_score(target,predicted)
    pre_svm=precision_score(target,predicted)
    f1_svm=f1_score(target,predicted)
    rec_svm=recall_score(target,predicted)



    print("--------------Classifier SVM--------------")
    print("Accuracy: "+str(acc_svm))
    print("Precision: "+str(pre_svm))
    print("Recall: "+str(rec_svm))
    print("F1: "+str(f1_svm))
    print("")




    modelRandomForest = RandomForestClassifier(random_state=90)
    modelRandomForest.fit(datos, labels)




    predicted_ceros=np.array(modelRandomForest.predict(X_unlabeled_ceros_valid))
    target_ceros=np.zeros_like(predicted_ceros)


    predicted_unos=np.array(modelRandomForest.predict(X_unlabeled_unos_valid))
    target_unos=np.ones_like(predicted_unos)



    predicted=np.concatenate((predicted_ceros,predicted_unos))
    target=np.concatenate((target_ceros,target_unos))

    acc_rf=accuracy_score(target,predicted)
    pre_rf=precision_score(target,predicted)
    f1_rf=f1_score(target,predicted)
    rec_rf=recall_score(target,predicted)

    print("-------------- Classifier Random Forest--------------")
    print("Accuracy: "+str(acc_svm))
    print("Precision: "+str(pre_svm))
    print("Recall: "+str(rec_svm))
    print("F1: "+str(f1_svm))
    print("")
    return (acc_svm,pre_svm,rec_svm,f1_svm),(acc_rf,pre_rf,rec_rf,f1_rf)

In [ ]:
def label_prop(data,target,fold=0,th=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target,fold=fold)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  label_prop_model = LabelPropagation(kernel='knn')
  label_prop_model.fit(datos, labels)

###################ZEROS#################
  pred_ceros=label_prop_model.predict_proba(X_unlabeled_ceros)

  prob_ceros=np.max(pred_ceros,axis=1)
  clases_ceros=np.argmax(pred_ceros,axis=1)

  ind_dudosos_ceros=np.where(prob_ceros<th)[0]
  ind_no_dudosos_ceros=np.where(prob_ceros>=th)[0]


  unos=np.where(clases_ceros==1)[0]

  ceros=np.where(clases_ceros==0)[0]


  unos=unos[np.isin(unos,ind_no_dudosos_ceros)]
  mal=unos.shape[0]
  ceros=ceros[np.isin(ceros,ind_no_dudosos_ceros)]
  bien=ceros.shape[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  dudoso_cero=np.array(X_unlabeled_ceros)[ind_dudosos_ceros]
  dudoso=ind_dudosos_ceros.shape[0]


######################ONES#############################################
  pred_unos=label_prop_model.predict_proba(X_unlabeled_unos)

  prob_unos=np.max(pred_unos,axis=1)
  clases_unos=np.argmax(pred_unos,axis=1)

  ind_dudosos_unos=np.where(prob_unos<th)[0]
  ind_no_dudosos_unos=np.where(prob_unos>=th)[0]


  unos=np.where(clases_unos==1)[0]

  ceros=np.where(clases_unos==0)[0]


  unos=unos[np.isin(unos,ind_no_dudosos_unos)]
  bien+=unos.shape[0]
  ceros=ceros[np.isin(ceros,ind_no_dudosos_unos)]
  mal+=ceros.shape[0]


  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]


  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  dudoso_uno=np.array(X_unlabeled_unos)[ind_dudosos_unos]

  dudosos=np.concatenate((dudoso_cero,dudoso_uno))
  dudoso+=ind_dudosos_unos.shape[0]

  metrics_svm,metrics_rf=comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (bien, mal,dudoso, nuevos_puntos_cero, nuevos_puntos_uno, dudoso,metrics_svm,metrics_rf)

In [ ]:
path=''
with open(path+'BCI.csv', 'w', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(['Method','acc_anot_1','%lab_1','svm_1_acc','svm_1_pre','svm_1_rec','svm_1_f1','rf_1_acc','rf_1_pre','rf_1_rec','rf_1_f1','acc_anot_2','%lab_2','svm_2_acc','svm_2_pre','svm_2_rec','svm_2_f1','rf_2_acc','rf_2_pre','rf_2_rec','rf_2_f1','acc_anot_3','%lab_3','svm_3_acc','svm_3_pre','svm_3_rec','svm_3_f1','rf_3_acc','rf_3_pre','rf_3_rec','rf_3_f1','acc_anot_4','%lab_4','svm_4_acc','svm_4_pre','svm_4_rec','svm_4_f1','rf_4_acc','rf_4_pre','rf_4_rec','rf_4_f1','acc_anot_5','%lab_5','svm_5_acc','svm_5_pre','svm_5_rec','svm_5_f1','rf_5_acc','rf_5_pre','rf_5_rec','rf_5_f1'])

In [ ]:
k=5

In [ ]:
fila=['Rocio_00']
for i in range(k):
  print("-------------------ITERATION "+str(i)+"---------------------")
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos_rocio(data,target,analizar_punto_rocio,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'BCI.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

In [ ]:
fila=['Rocio_08']
for i in range(k):
  print("-------------------ITERATION "+str(i)+"---------------------")
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos_rocio(data,target,analizar_punto_rocio,th=0.8,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'BCI.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

In [ ]:
fila=['Rocio_06']
for i in range(k):
  print("-------------------ITERATION "+str(i)+"---------------------")
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos_rocio(data,target,analizar_punto_rocio,th=0.6,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'BCI.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

In [ ]:
k=5

# Bottleneck threshold

In [ ]:
fila=['Bottleneck_08']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos(data,target,analizar_punto_th,0.8,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'BCI.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# Wasserstein threshold

In [ ]:
fila=['Wasserstein_08']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.8,fold=i)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'BCI.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)

# LabelPropagation 0.8

In [ ]:
fila=['LabelPropagation_08']
for i in range(k):
  (data,target) = load_prima()
  (bien,mal,dudoso,pcero,puno,pdudoso,acc_svm,acc_rf)=label_prop(data,target,fold=i,th=0.8)
  acc,anotados=resumen(bien,mal,dudoso)
  print("")
  print("")
  fila.append(acc)
  fila.append(anotados)
  fila.append(acc_svm[0])
  fila.append(acc_svm[1])
  fila.append(acc_svm[2])
  fila.append(acc_svm[3])
  fila.append(acc_rf[0])
  fila.append(acc_rf[1])
  fila.append(acc_rf[2])
  fila.append(acc_rf[3])
with open(path+'BCI.csv', 'a', newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  spamwriter.writerow(fila)